# Face Recognition using LBPH

Feature extraction used this library: https://github.com/bytefish/facerec

In [1]:
from facerec.feature import SpatialHistogram
from PIL import Image
import numpy as np

In [2]:
img = Image.open('../face_data/1_0.png').convert('L')

In [3]:
feature_extractor = SpatialHistogram()

In [4]:
feature = feature_extractor.extract(img)

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/facerec/feature.py:266: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  H = np.histogram(C, bins=2**self.lbp_operator.neighbors, range=(0, 2**self.lbp_operator.neighbors), normed=True)[0]


In [5]:
feature.shape

(16384,)

## Define Class for Feature Extraction

In [6]:
from facerec.feature import SpatialHistogram
from PIL import Image

class FeatureExtractor:
    def __init__(self):
        print('Init')
        self.lbph_extractor = SpatialHistogram()
        self.img_size = (60, 60)
        
    def extract_lbph(self, gray_img):
        return self.lbph_extractor.extract(gray_img)
        
    def extract_lbph_batch(self, img_path_list):
        features = []
        for img_path in img_path_list:
            gray_img = Image.open(img_path).convert('L')
            gray_img = gray_img.resize(self.img_size, Image.ANTIALIAS)
            features.append(self.extract_lbph(gray_img))
        return features

## Read The Data

In [7]:
import os

In [8]:
img_path_list = os.listdir('../face_data')
img_path_list = [os.path.join('../face_data', img_path) for img_path in img_path_list]

In [9]:
target = []
for img_path in img_path_list:
    if ('/1_' in img_path):
        target.append(1)
    elif ('/2_' in img_path):
        target.append(2)
    elif ('/7_' in img_path):
        target.append(3)
    elif ('/12_' in img_path):
        target.append(3)
    elif ('/17_' in img_path):
        target.append(4)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train_img_path_list, X_test_img_path_list, y_train, y_test = train_test_split(
    img_path_list, target, test_size=0.2, random_state=0
)

In [12]:
feature_extractor = FeatureExtractor()

Init


In [13]:
%%time
X_train_histogram = feature_extractor.extract_lbph_batch(X_train_img_path_list)
X_test_histogram = feature_extractor.extract_lbph_batch(X_test_img_path_list) 

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/facerec/feature.py:266: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  H = np.histogram(C, bins=2**self.lbp_operator.neighbors, range=(0, 2**self.lbp_operator.neighbors), normed=True)[0]


CPU times: user 3.27 s, sys: 42.4 ms, total: 3.31 s
Wall time: 3.31 s


## Predict using Euclidean Distance

In [14]:
from scipy.spatial.distance import euclidean

In [15]:
def predict(image_representation_list, true_label, test_representation):
    minimum_label = None
    minimum_distance = np.inf
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

In [16]:
predict(X_train_histogram, y_train, X_test_histogram[0])

4

In [17]:
predicted_list = []
for test_histogram in X_test_histogram:
    predicted_list.append(predict(X_train_histogram, y_train, test_histogram))

In [18]:
from sklearn.metrics import accuracy_score, f1_score

In [19]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

0.9824561403508771
0.9820793814601864


## Predict using Downloaded Dataset